In [115]:

import json
import pandas as pd

In [116]:
with open('.\\output\\Custody_form.json', 'r', encoding='utf-8') as t:
    data = json.load(t)  #data

In [113]:

# Extract form fields
def extract_form_fields(data, confidence_treshold):
    pages = data['document']['pages']
    extracted_form_fields = []
    
    for page in pages:
        form_fields = page.get('formFields', [])
        
        for form_field in form_fields:
            field_name = form_field.get('fieldName', {}).get('textAnchor', {}).get('content', '').strip()
            field_value = form_field.get('fieldValue', {}).get('textAnchor', {}).get('content', '').strip()
            field_confidence = form_field.get('fieldName', {}).get('confidence', {})
            if field_confidence > confidence_treshold:
                extracted_form_fields.append((field_name, field_value, field_confidence))

    return extracted_form_fields

# Extract entity types
def extract_entity_types(data):
    entity_types = []
    for entity in data['document']['entities']:
        for property in entity['properties']:
            entity_type = property['type']
            mention_text = property['mentionText']
            confidence = property['confidence']
            entity_types.append({
                'entityType': entity_type,
                'mentionText': mention_text,
                'confidence': confidence
            })
    return entity_types



entity_types = extract_entity_types(data) #{'entityType':'phone', 'mentionTex': '060881525', 'confidence': 0.5874}
extracted_data = extract_form_fields(data, 0.5)
field_value_df = pd.DataFrame()
field_value_df['Field'] = [x for x,_,_ in extracted_data ]
field_value_df['value'] = [x for _,x,_ in extracted_data ]
field_value_df['confidence'] = [x for _,_,x in extracted_data ]


field_value_df['data_type'] = None
field_value_df['data_type_confidence'] = None

for index, row in field_value_df.iterrows():
    field_text = str(row['Field']).lower()
    value_text = str(row['value'])
    field_confidence = row['confidence']
    max_confidence =[]
    for ent in entity_types:
        entity_type = ent['entityType']
        entity_text = ent['mentionText']
        entity_confidence = ent['confidence']
    
        # Check if the entity type is in 'Field' or 'value' (case-insensitive)
        if (entity_text in field_text or entity_text in value_text) and entity_type not in ('page_number') :
            max_confidence.append(ent)
    try:
        if max_confidence:
            max_entity = max(max_confidence, key=lambda x: x['confidence'])
            field_value_df.at[index, 'data_type'] = max_entity['entityType']
            field_value_df.at[index, 'data_type_confidence'] = max_entity['confidence']
    except ValueError:
        pass

    
field_value_df
excel_file = '.\\output\\Security_document_tables2.xlsx'
field_value_df.to_excel(excel_file, sheet_name='field - values', index=False, na_rep='None')

,Field,value,confidence,data_type,data_type_confidence
0,LINE,12.,0.601466,None,None
1,LINE,13,0.589684,None,None
2,DEFENDANT,Emily Brath,0.997030,person,0.995271
3,PLAINTIFF,Mouad Maaziz,0.996503,person,0.98806
4,DOB:,05/12/2020,0.966626,date_time,0.995292
5,DOB:,05/05/2018,0.965952,date_time,0.995822
6,Address:,"Burlington, Ontario L7R 2G6",0.936856,address,0.931318
7,Name:,Ismaeil Maaziz,0.934523,person,0.967978
8,"resides at (street, city, state, zip)","789 Elm St, Othertown, Province 98765.",0.907822,address,0.993175
9,Address:,"Burlington, Ontario L7R 2G6",0.886250,address,0.931318


In [98]:
def get_field_value(json_file):
    "Extracting Fields and Values from the JSON file"

    extracted_form_fields = []
    field_value_df = pd.DataFrame()
    pages = json_file['document']['pages']
    for page in pages:
        form_fields = page.get('formFields', [])
        
        for form_field in form_fields:
            field_name = form_field.get('fieldName', {}).get('textAnchor', {}).get('content', '').strip()
            field_value = form_field.get('fieldValue', {}).get('textAnchor', {}).get('content', '').strip()
            
            extracted_form_fields.append( (field_name, field_value))
    field_value_df['Field'] = [x for x,_ in extracted_form_fields]
    field_value_df['value'] = [x for _,x in extracted_form_fields]
    
    return field_value_df
get_field_value(data)


,Field,value
0,Detailed Earnings information\n(If you check t...,☑
1,Yes,☐
2,No,☑
3,Date of Birth\n(Mo/Day/Yr),03/01/1997
4,Date,19/10/2023
5,For the period(s)/year(s):,2020-2022
6,Name,Mouad Maaziz
7,Number,555-66-7777
8,Address,"789 Elm St, Othertown, Province 98765."
9,Name,Mustafa B FAKAK


,Field,value,confidence,data_type,data_type_confidence
0,Detailed Earnings information\n(If you check t...,☑,0.999676,None,None
1,Yes,☐,0.999382,None,None
2,No,☑,0.992163,None,None
3,Date of Birth\n(Mo/Day/Yr),03/01/1997,0.914074,date_time,0.995005
4,Date,19/10/2023,0.874462,date_time,0.996514
5,For the period(s)/year(s):,2020-2022,0.815732,id,0.673699
6,Name,Mouad Maaziz,0.786341,person,0.888976
7,Number,555-66-7777,0.719963,id,0.989122
8,Address,"789 Elm St, Othertown, Province 98765.",0.709777,address,0.990868
9,Name,Mustafa B FAKAK,0.665716,person,0.846234


In [76]:
# Extract entity types
def extract_entity_types(data):
    entity_types = []
    for entity in data['document']['entities']:
        for property in entity['properties']:
            entity_type = property['type']
            mention_text = property['mentionText']
            confidence = property['confidence']
            entity_types.append({
                'entityType': entity_type,
                'mentionText': mention_text,
                'confidence': confidence
            })

    # Print or process the extracted entity types
    for entity in entity_types:
        print(f"Entity Type: {entity['entityType']}")
        print(f"Mention Text: {entity['mentionText']}")
        print(f"Confidence: {entity['confidence']}")
        print()

In [77]:
def combine_extraction(data):
    form_fields = extract_form_fields(data)
    entity_types = extract_entity_types(data)

    # You can perform further processing or analysis using both form fields and entity types

    return form_fields, entity_types
print(combine_extraction(data))

Entity Type: quantity
Mention Text: 342.01 PSI
Confidence: 0.97277623

Entity Type: page_number
Mention Text: 17 of 20
Confidence: 0.9935057

Entity Type: date_time
Mention Text: March 1, 2012
Confidence: 0.9928075

Entity Type: id
Mention Text: 555-66-7777
Confidence: 0.98912245

Entity Type: person
Mention Text: Mouad Maaziz
Confidence: 0.8889764

Entity Type: date_time
Mention Text: 03/01/1997
Confidence: 0.9950053

Entity Type: id
Mention Text: 2020-2022
Confidence: 0.6736993

Entity Type: phone
Mention Text: 1-800-772-1213
Confidence: 0.99956685

Entity Type: page_number
Mention Text: 3
Confidence: 0.58820766

Entity Type: price
Mention Text: $ 500,000
Confidence: 0.9680597

Entity Type: price
Mention Text: $15.00
Confidence: 0.98402756

Entity Type: price
Mention Text: $
$300,500
Confidence: 0.6153776

Entity Type: price
Mention Text: $5,000
Confidence: 0.97045946

Entity Type: quantity
Mention Text: one year
Confidence: 0.57983744

Entity Type: email
Mention Text: mouad.maziz@gm

In [114]:
def get_tables(json_file ) :
    """Extracting tables from the all the pages in the document"""
    
    document = json_file['document']
    text = document['text']
    print(f"Full document text: {repr(text)}\n")
    print(f"There are {len(document['pages'])} page(s) in this document.")

    # Read the form fields and tables output from the processor
    for page in document['pages']:
        print(f"\n\n**** Page {page['pageNumber']} ****")

        print(f"\nFound {len(page['tables'])} table(s):")

        for idx, table in enumerate(page['tables']):
            num_columns = len(table['headerRows'][0]['cells'])
            num_rows = len(table['bodyRows'])
            print(f"Table with {num_columns} columns and {num_rows} rows:")

            # Print header rows
            print("Columns:")
            print_table_rows(table['headerRows'], text)
            # Print body rows
            print("Table body data:")
            print_table_rows(table['bodyRows'], text)

            # Create a DataFrame for the table
            table_data = []
            table_data.append([layout_to_text(cell['layout'], text) for cell in table['headerRows'][0]['cells']])
            table_data.extend([layout_to_text(cell['layout'], text) for cell in row['cells']] for row in table['bodyRows'])
            table_df = pd.DataFrame(table_data)

    return table_df

def print_table_rows(table_rows, text) -> None:
    for table_row in table_rows:
        row_text = ""
        for cell in table_row['cells']:
            cell_text = layout_to_text(cell['layout'], text)
            row_text += f"{repr(cell_text.strip())} | "
        print(row_text)

def layout_to_text(layout , text: str) -> str:
    """
    Document AI identifies text in different parts of the document by their
    offsets in the entirety of the document"s text. This function converts
    offsets to a string.
    """
    # If a text segment spans several lines, it will
    # be stored in different text segments.
    return "".join(
        text[int(segment['startIndex']) : int(segment['endIndex'])]
        for segment in layout['textAnchor']['textSegments']
    )


get_tables(data)

Full document text: 'COMPLAINT FOR CUSTODY\nINSTRUCTION SHEET\nUSE THIS FORM IF THERE IS NO CUSTODY ORDER.\nThese instructions are meant to give you\ngeneral information and not legal advice.\n1. Fill in the Domestic Relations Information Sheet with as much information as you have.\n2. Complete, date, and sign the Complaint for Custody (detailed instructions included).\n3. The filing fee for a complaint for custody is $102.38. If you cannot pay the filing fee, you\nmay ask to be excused from paying the fee by filing a Petition to Proceed In Forma Pauperis\n(IFP). If you receive public assistance or SSI, bring your public assistance photo ID or proof that\nyou receive SSI.\n4. File the completed complaint and information sheet with the filing fee by mailing or hand-\ndelivering them in person to:\nClerk of Family Court\n1501 Arch Street-11th Floor\nPhiladelphia, Pa. 19102\n5. If you file in person, you may pay the filing fee by money order, or credit card. If you\nfile by mail, you may 

,0
0,is:\nNot allowed to say\n
